В данном примере будет видно что БПФ хорошо справляется с сигналами в которых частоты присутствуют одновременно и их присутствие не обусловлено зависимостью от времени.

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot
%run ../lab-3/lab-3-spectrogram.ipynb

In [ ]:
n = 2000
n_4 = n // 4
t = np.linspace(0., 2 * np.pi, n, endpoint=False)

y_0 = 0.
y_1 = 3. * np.cos(5 * t)
y_2 = 7. * np.cos(13 * t)
y_3 = 5. * np.cos(19 * t)
y_4 = 11. * np.cos(31 * t)

y_static = y_0 + y_1 + y_2 + y_3 + y_4

Сигнал состоит из 4 частот. На этом сигнале все частоты присутствуют одновременно.

In [ ]:
matplotlib.pyplot.plot(t, y_static)

После БПФ видим эти частоты в частотной области сигнала.

In [ ]:
df_static = np.fft.fft(y_static)
matplotlib.pyplot.plot(np.abs(df_static)[:40], '--')

Теперь представим сигнал в котором наличие частот зависит от времени. Ниже представлен сигнал в котором писутствуют те же частоты однако они присутствуют в сигнале не одновременно, а в разное время по одной частоте на каждом отрезке времени.

In [ ]:
y_dynamic = np.concatenate([y_1[0*n_4:1*n_4], y_2[1*n_4:2*n_4], y_3[2*n_4:3*n_4], y_4[3*n_4:4*n_4]])
matplotlib.pyplot.plot(y_dynamic)
assert len(y_dynamic) == len(y_static)

В частотной области этого сигнала невозможно явно оперделить какие частоты присутствуют в сигнале. На рисунке в принципе видны пики на соответствующих частотах, однако картина получается размытой.

In [ ]:
df_dynamic = np.fft.fft(y_dynamic)
matplotlib.pyplot.plot(np.abs(df_dynamic)[:40], '--')

Можно посмотреть на частотный спектр обих сигналов одновременно с нормализацией.

In [ ]:
df_static_max = np.amax(np.abs(df_static))
df_dynamic_max = np.amax(np.abs(df_dynamic))

matplotlib.pyplot.plot(np.abs(df_static)[:40] / df_static_max, '--')
matplotlib.pyplot.plot(np.abs(df_dynamic)[:40] / df_dynamic_max, '--')

Для анализа динамического сигнала можно использовать оконное преобразование Фурье. Результат оконного преобразования Фурье это двумерная картинка в которой во горизонтальной оси отложено время, а по вертикальной оси частоты. Цветом кодируется интенсивность. Таким образом оконное преобразование Фурье показывает эволюцию частот по времени. На картинках ниже представлено оконное преобразование Фурье.

In [ ]:
s_1 = spectrogram(y_dynamic, 2048, 1.0)
s_2 = spectrogram(y_dynamic, 2048, 500.0)

fig, ax = matplotlib.pyplot.subplots(2,1)

#ax[0].set_title('Широкое окно - неопределённость по времени')
ax[0].matshow(np.abs(s_1).T[:100,:], aspect='auto')

#ax[1].set_title('Узкое окно - неопределённость по частотам')
ax[1].matshow(np.abs(s_2).T[:100,:], aspect='auto')

Оконное преобразование Фурье имеет недостаток который характеризуется неопределённостью времени и частоты. Чем точнее на спектрограмме время, тем расплывчатее частоты (второй рисунок) и наоборот чем точнее частоты тем менее определённое время (первый рисунок).

Для того чтобы анализировать такие динамические сигналы используют вейвлеи анализ. В питоне есть библиотека __pywt__ для работы с вейвлетами и модуль __scaleogram__ для построения скалограмм.

In [ ]:
import pywt
import scaleogram as scg

Народобие спектрограммы которая получается с помощью оконного преобразования Фурье, вейвлет преобразование также можно рассмотреть на двумерной плоскости. Такой рисунок называется __скалограммой__. На скалограмме по горизонтале отложено время (точно также как на спектрограмме) однако по вертикале отложен фактор растяжения вейвлета.

In [ ]:
scg.set_default_wavelet('cmor1-1.5')
#scg.set_default_wavelet('cgau5')
#scg.set_default_wavelet('cgau1')
#scg.set_default_wavelet('shan0.5-2')
#scg.set_default_wavelet('mexh')

In [ ]:
scales = scg.periods2scales(np.arange(1, 300))
#scales = scg.periods2scales([5, 13, 19, 31])
scg.cws(y_dynamic, scales=scales); # figsize=(6.9,2.3)

На картинке выше видно явно выделенные временные домены в которых присутствует только один вейвлет.

In [ ]:
scales = scg.periods2scales(np.arange(1, 300))
scg.cws(y_static, scales=scales);

Тот же, но статический сигнал. На картинке видно что в каждый момент времени сигнал состоит из нескольких вейвлетов одновременно в отличие от предыдущей картинки.

В качестве примера был использован материал:

[1] http://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

[2] https://blog.octo.com/en/time-series-features-extraction-using-fourier-and-wavelet-transforms-on-ecg-data/

[3] https://www.kaggle.com/asauve/a-gentle-introduction-to-wavelet-for-data-analysis/notebook?scriptVersionId=12579739#Introduction